In [121]:
#libraries and tool importation
import numpy as np # linear algebra
import matplotlib.pylab as plt
import pandas as pd
import pandas as pd # data processing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dropout
import tensorflow as tf 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from keras.layers import BatchNormalization, Dropout
from keras.layers import Dense, Activation
from tensorflow import keras

from sklearn.neighbors import KNeighborsClassifier  
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

plt.rcParams["figure.figsize"] = (16,10) #Make the plots bigger by default
plt.rcParams["lines.linewidth"] = 2 #Setting the default line width
plt.style.use("ggplot")

In [144]:
#original emg data
df = pd.read_csv("/Users/dhruvaggarwal/Development/EEG-Classification/set_2_emg.csv")

In [82]:
df.dtypes

Unnamed: 0      int64
DATE           object
TIME           object
EMG 1           int64
EMG 2         float64
EMG 3         float64
EMG 4         float64
EMG 5         float64
open            int64
closed          int64
thumbs up       int64
victory         int64
swag            int64
yoga            int64
shooting        int64
salute          int64
dtype: object

In [83]:
# Assuming df is your pandas DataFrame
# You can replace "df" with the actual variable name of your DataFrame

# Remove rows where both "open" and "close" columns are zero
df_filtered = df[(df['open'] != 0) | (df['closed'] != 0) | (df['thumbs up'] != 0)| (df['victory'] != 0)| (df['swag'] != 0)| (df['yoga'] != 0)| (df['shooting'] != 0)| (df['salute'] != 0)]

# Reset the index after filtering
df_filtered = df_filtered.reset_index(drop=True)


In [148]:

# Combine the four class columns into a single column 'class_label'
class_columns = ['open', 'closed', 'thumbs up', 'victory','swag','yoga','shooting','salute']
df['class_label'] = df[class_columns].idxmax(axis=1)
df['class_label'] = df['class_label'].apply(lambda x: class_columns.index(x))

In [149]:
df

,Unnamed: 0,DATE,TIME,EMG 1,EMG 2,EMG 3,EMG 4,EMG 5,open,closed,thumbs up,victory,swag,yoga,shooting,salute,class_label
0,0,8-27-2023,17:48:49,0,0.0,0.0,0.0,NaN,0,1,0,0,0,0,0,0,1
1,1,8-27-2023,17:48:49,0,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,0,1
2,2,8-27-2023,17:48:49,-1,-1.0,0.0,0.0,2.0,0,1,0,0,0,0,0,0,1
3,3,8-27-2023,17:48:49,2,-5.0,-6.0,4.0,8.0,0,1,0,0,0,0,0,0,1
4,4,8-27-2023,17:48:49,-1,-5.0,0.0,0.0,-1.0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217,1217,8-27-2023,17:52:55,0,1.0,0.0,-2.0,0.0,1,0,0,0,0,0,0,0,0
1218,1218,8-27-2023,17:52:55,2,0.0,-2.0,0.0,2.0,1,0,0,0,0,0,0,0,0
1219,1219,8-27-2023,17:52:55,0,-1.0,0.0,0.0,0.0,1,0,0,0,0,0,0,0,0
1220,1220,8-27-2023,17:52:55,0,-1.0,-1.0,2.0,0.0,1,0,0,0,0,0,0,0,0


In [151]:
df.dtypes

Unnamed: 0       int64
DATE            object
TIME            object
EMG 1            int64
EMG 2          float64
EMG 3          float64
EMG 4          float64
EMG 5          float64
open             int64
closed           int64
thumbs up        int64
victory          int64
swag             int64
yoga             int64
shooting         int64
salute           int64
class_label      int64
dtype: object

In [152]:
x = df.loc[:,["EMG 1", "EMG 2","EMG 3","EMG 4","EMG 5"]]
# y = df.loc[:,4:7]

In [153]:
x

,EMG 1,EMG 2,EMG 3,EMG 4,EMG 5
0,0,0.0,0.0,0.0,NaN
1,0,NaN,NaN,NaN,NaN
2,-1,-1.0,0.0,0.0,2.0
3,2,-5.0,-6.0,4.0,8.0
4,-1,-5.0,0.0,0.0,-1.0
...,...,...,...,...,...
1217,0,1.0,0.0,-2.0,0.0
1218,2,0.0,-2.0,0.0,2.0
1219,0,-1.0,0.0,0.0,0.0
1220,0,-1.0,-1.0,2.0,0.0


In [154]:
y = df.loc[:,['class_label']]

In [155]:
y

,class_label
0,1
1,1
2,1
3,1
4,1
...,...
1217,0
1218,0
1219,0
1220,0


In [156]:
#check of the dimensions
print(x.shape) 
print(y.shape) 
print(df.shape[0])

(1222, 5)
(1222, 1)
1222


In [106]:
y.dtypes

open         int64
closed       int64
thumbs up    int64
victory      int64
swag         int64
yoga         int64
shooting     int64
salute       int64
dtype: object

In [53]:
x.shape[1]

2

In [157]:
import pandas as pd

# Assuming x is your input data
x = x.fillna(0)


In [158]:
x

,EMG 1,EMG 2,EMG 3,EMG 4,EMG 5
0,0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0
2,-1,-1.0,0.0,0.0,2.0
3,2,-5.0,-6.0,4.0,8.0
4,-1,-5.0,0.0,0.0,-1.0
...,...,...,...,...,...
1217,0,1.0,0.0,-2.0,0.0
1218,2,0.0,-2.0,0.0,2.0
1219,0,-1.0,0.0,0.0,0.0
1220,0,-1.0,-1.0,2.0,0.0


In [159]:
y

,class_label
0,1
1,1
2,1
3,1
4,1
...,...
1217,0
1218,0
1219,0
1220,0


In [160]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the Random Forest model
model = RandomForestClassifier(n_estimators=100,random_state=42)
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')

Accuracy: 0.18775510204081633
Classification Report:
              precision    recall  f1-score   support

           0       0.33      0.43      0.38        21
           1       0.16      0.17      0.16        24
           2       0.44      0.32      0.38        37
           3       0.18      0.11      0.14        37
           4       0.12      0.15      0.13        27
           5       0.08      0.11      0.10        27
           6       0.12      0.08      0.10        37
           7       0.14      0.20      0.17        35

    accuracy                           0.19       245
   macro avg       0.20      0.20      0.19       245
weighted avg       0.20      0.19      0.19       245



/var/folders/m9/nn6zgszd65v8mmskfjh44b_r0000gn/T/ipykernel_53674/679602326.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_scaled, y_train)
